In [1]:
# imports
from __future__ import annotations

import math
import requests
import xml.etree.ElementTree as ET

In [2]:
API_KEY = "0AaKvkxb1Q452IX0NDW0zmN9TBKu1gquS5zoll4d5Ws"
API_URL = "https://atlas.microsoft.com/weather/route/json"
SPEED = 55

In [3]:
class Coordinate:
    def __init__(self, lon: float, lat: float, elevation: float):
        self.lon = lon
        self.lat = lat
        self.elevation = elevation

    def __str__(self):
        return f"Lat: {self.lat} | Lon: {self.lon} | Elevation: {self.elevation}"
    
    def __repr__(self):
        return f"Lat: {self.lat} | Lon: {self.lon} | Elevation: {self.elevation}"

In [4]:
class Placemark:
    def __init__(self, name: str, coords: list[Coordinate]):
        self.name = name
        self.coords = coords

    def __str__(self):
        return f"{self.name}: {self.coords}"
    
    def __repr__(self):
        return f"{self.name}: {self.coords}"

In [5]:
def parse_kml_file(filename):
    root = ET.parse(filename).getroot()
    placemarks = []
    for child in root[0]:
        if child.tag == '{http://www.opengis.net/kml/2.2}Placemark':
            # name tag
            name = child.find('{http://www.opengis.net/kml/2.2}name').text
            coords = []
            # coordinates tag
            p_coords = child.find('{http://www.opengis.net/kml/2.2}LineString').find('{http://www.opengis.net/kml/2.2}coordinates')
            for line in p_coords.text.split("\n"):
                line = line.strip() 
                if (len(line) == 0):
                    continue

                parts = line.split(",")
                coords.append(Coordinate(float(parts[0]),float(parts[1]),float(parts[2])))

            placemarks.append(Placemark(name, coords))


    return placemarks

In [6]:
def batch_elems(elems: list, size: int):
    if (len(elems) == 0):
        return []
    
    out = [[]]

    for elem in elems:
        if len(out[-1]) < size:
            out[-1].append(elem)
        else:
            out.append([elem])

    return out

In [7]:
#equation to calculate distance between coordinates 
def heversine(lon1: float, lat1: float, lon2: float, lat2: float):
    R = 6371.0  #Radius of the Earth in km
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    
    return distance

In [8]:
def flat_2d(input: list[list]):
    out = []
    for i in input:
        out.extend(i)

    return out

In [9]:
def weather_data_req(data: list[tuple[Coordinate, float]]):
    query = ":".join(map(lambda d: f"{d[0].lat},{d[0].lon},{d[1]}", data))
    
    params = {
        "api-version": "1.1",
        "subscription-key": API_KEY,
        "query": query
    }

    res = requests.get(API_URL, params)
    if not (res.status_code == 200):
        raise Exception(res)

    # json = res.json()
    # assert(len(json["waypoints"]) == len(data))
    return res

In [10]:
def get_weather_data(coords: list[Coordinate], time_offset: int = 0, speed: float = SPEED):
    if len(coords) == 0: return []

    coords_with_time: list[tuple[Coordinate, float]] = []
    running_distance = 0

    for i in range(len(coords)):
        c1 = coords[i]
        if i > 0:
            c1 = coords[i-1]
            
        c2 = coords[i]
        distance = heversine(float(c1.lon), float(c1.lat), float(c2.lon), float(c2.lat))
        running_distance = running_distance + distance
        
        coords_with_time.append(
            (
                c2,
                min((running_distance / speed * 60)+time_offset, 120) # max 120 minutes
            )
        )
    
    # list[list[tuple[Coordinate, float]]]
    # [
    #     [
    #         (coord, eta),
    #         (coord, eta),
    #         (coord, eta),
    #         ...
    #     ],
    #     [
    #         (coord, eta),
    #         (coord, eta),
    #         (coord, eta),
    #         ...
    #     ],
    #     ...
    # ]
    batched: list[list[tuple[Coordinate, float]]] = batch_elems(coords_with_time, 60)

    responses = []

    for batch in batched:
        res = weather_data_req(batch)
        if not (res.status_code == 200):
            print(res.json())
            raise Exception(res.json())
        responses.append(res.json())

    # return list of waypoints
    return flat_2d(map(lambda r: r["waypoints"], responses))

In [15]:
# data: [(distance, elevation, cloud_cover_%, wind_dir, wind_speed)]
def write_csv(name: str, data: list[tuple[float, float, float, float, int, int, int]]):
    path = f"data/generated/{name}.csv"

    with open(path, 'w') as csvfile:
        csvfile.write(f'{name}\nLat,Lon,Distance,Elevation,Cloud Cover,Wind Direction,Wind Speed\n')
        for row in data:
            csvfile.write(f"{row[0]},{row[1]},{row[2]},{row[3]},{row[4]},{row[5]},{row[6]}\n")

In [16]:
def calc_distance(coords: list[Coordinate], current_coord: Coordinate):
    running_total = 0
    stop_index = coords.index(current_coord)
    for i in range(stop_index+1):
       if i > 0: running_total += heversine(coords[i-1].lon, coords[i-1].lat, coords[i].lon, coords[i].lat)
    return running_total

In [17]:
def generate_data(path_to_kml: str):
    placemarks = parse_kml_file(path_to_kml)

    for placemark in placemarks:
        route_data: list[tuple[float, float, float, float]] = list(map(lambda p: (p.lat, p.lon, calc_distance(placemark.coords, p), p.elevation), placemark.coords))
        weather_data: list[tuple[int, int ,int]] = list(map(lambda w: (w["cloudCover"], w["wind"]["direction"]["degrees"], w["wind"]["speed"]["value"]), get_weather_data(placemark.coords)))

        assert(len(weather_data) == len(route_data))
        
        csv_data: list[tuple[float, float, float, float, int, int, int]] = []
        for i in range(len(route_data)):
            csv_data.append((route_data[i][0],route_data[i][1], route_data[i][2], route_data[i][3], weather_data[i][0], weather_data[i][1], weather_data[i][2]))
            
        write_csv(placemark.name, csv_data)
        print(placemark.name)

# (condition) ? val1 : val2

# val1 if condition else val2

In [18]:
generate_data("data/Main Route.kml")

A. Independence to Topeka
C. Grand Island to Gering
D. Gering to Casper
E. Casper to Lander
F. Lander to Montpelier
G. Montpelier to Pocatello
H. Pocatello to Twin Falls
B. Topeka to Grand Island
